In [1]:
import os 
import sys

import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import IPython.display as ipd

from scipy.io import wavfile as wav
from sklearn import metrics 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adam
from keras.utils import to_categorical

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
PATH = os.getcwd() + '/speech_commands_v0.01/'

In [3]:
def get_directory_contents(path):
    return os.listdir(path)

def open_file(filename):
    
    f = open(filename)
    return f.read().splitlines()

def compile_dataset(folders):
    
    total_words = []
    for folder in folders:
        words = get_directory_contents(path=PATH+folder)
        words = [folder+'/'+word for word in words]
        total_words = total_words + words
    
    dataset = create_df(words=total_words)
                           
    return dataset

def create_df(words):
    
    data = pd.DataFrame({'recordings':words})

    data['word'] = data['recordings'].str.split('/').str[0]
    data['speaker_id'] = data['recordings'].str.split('/').str[1]
    data['speaker_id'] =data['speaker_id'].str.split('_').str[0]
    
    return data

def summary(data):
    
    summary_df = pd.DataFrame()
    summary_df['total_recordings'] = [data.shape[0]]
    summary_df['total_speakers'] = len(data['speaker_id'].unique())
    summary_df['total_words'] = len(data['word'].unique())
    
    return summary_df

def word_distribution():
    
    word_count = data['word'].value_counts()
    ax = word_count.plot(kind='bar', figsize=(8,4), alpha=0.5)
    plt.show()

In [4]:
remove_words = [
    '.DS_Store', 'validation_list.txt', 'LICENSE',
    '_background_noise_', 'README.md', 'testing_list.txt'
]

names = get_directory_contents(path=PATH)
names = [word for word in names if word not in remove_words]

In [11]:
val_list = open_file(filename=PATH+'validation_list.txt')
val_df = create_df(words=val_list)
test_list = open_file(filename=PATH+'testing_list.txt')
test_df = create_df(words=test_list)
total_df = compile_dataset(folders=names)

training_df = total_df[~total_df['recordings'].isin(val_df['recordings'])]
training_df = training_df[~training_df['recordings'].isin(test_df['recordings'])]

In [26]:
training_df

,recordings,word,speaker_id
0,right/988e2f9a_nohash_0.wav,right,988e2f9a
1,right/1eddce1d_nohash_3.wav,right,1eddce1d
2,right/93ec8b84_nohash_0.wav,right,93ec8b84
3,right/6272b231_nohash_1.wav,right,6272b231
5,right/2f813234_nohash_1.wav,right,2f813234
...,...,...,...
64714,four/10c6d873_nohash_0.wav,four,10c6d873
64715,four/333784b7_nohash_1.wav,four,333784b7
64716,four/93f30cc4_nohash_1.wav,four,93f30cc4
64718,four/238c112c_nohash_0.wav,four,238c112c


In [28]:
fn = 'speech_commands_v0.01/four/10c6d873_nohash_0.wav'
librosa_audio, librosa_sample_rate = librosa.load(fn)
scipy_sample_rate, scipy_audio = wav.read(fn)
print("Original sample rate: {}".format(scipy_sample_rate))
print("Librosa sample rate: {}".format(librosa_sample_rate))

Original sample rate: 16000
Librosa sample rate: 22050


In [29]:
print('Original audio file min~max range: {} to {}'.format(np.min(scipy_audio), np.max(scipy_audio)))
print('Librosa audio file min~max range: {0:.2f} to {0:.2f}'.format(np.min(librosa_audio), np.max(librosa_audio)))

Original audio file min~max range: -32768 to 16883
Librosa audio file min~max range: -1.00 to -1.00


In [30]:
mfccs = librosa.feature.mfcc(y=librosa_audio, sr=librosa_sample_rate, n_mfcc = 40)

In [20]:
mfccs.shape

(40, 44)

In [25]:
mfccs.shape

(40, 44)

In [31]:
mfccs.shape

(40, 44)

In [32]:
def extract_features(file_name):
    audio, sample_rate = librosa.load(file_name) 
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_processed = np.mean(mfccs.T,axis=0)
     
    return mfccs_processed

In [54]:
def dataset_prep(df):
    features = []
    # Iterate through each sound file and extract the features 
    for index, row in df.iterrows():
        file_name = 'speech_commands_v0.01/'+row['recordings']
        class_label = row["word"]
        data = extract_features(file_name)

        features.append([data, class_label])
    # Convert into a Panda dataframe 
    featuresdf = pd.DataFrame(features, columns=['feature','class_label'])
    return featuresdf

In [56]:
# featuresdf = dataset_prep(df)
# featuresdf.to_csv("training_set.csv")

testdf = dataset_prep(df=test_df)
valdf = dataset_prep(df=val_df)

In [57]:
testdf.to_csv("training_set.csv")
valdf.to_csv("validation_set.csv")

In [38]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [58]:
# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())
# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y))

x_val = np.array(valdf.feature.tolist())
y_val = np.array(valdf.class_label.tolist())
y_val = to_categorical(le.transform(y_val))

x_test = np.array(testdf.feature.tolist())
y_test = np.array(testdf.class_label.tolist())
y_test = to_categorical(le.transform(y_test))

In [51]:
num_labels = yy.shape[1]
filter_size = 2
def build_model_graph(input_shape=(40,)):
    model = Sequential()
    model.add(Dense(30))
    model.add(Activation('relu'))
    # Compile the model
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    return model
model = build_model_graph()

In [59]:
num_epochs = 100
num_batch_size = 32
model.fit(X, yy, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_val, y_val), verbose=1)

Epoch 1/100
1597/1597 [==============================] - 2s 1ms/step - loss: 6.1674 - accuracy: 0.1005 - val_loss: 6.1395 - val_accuracy: 0.1175
Epoch 2/100
1597/1597 [==============================] - 2s 1ms/step - loss: 6.0702 - accuracy: 0.1304 - val_loss: 6.0711 - val_accuracy: 0.1802
Epoch 3/100
1597/1597 [==============================] - 2s 1ms/step - loss: 6.0262 - accuracy: 0.1564 - val_loss: 6.0418 - val_accuracy: 0.17120s -
Epoch 4/100
1597/1597 [==============================] - 2s 1ms/step - loss: 6.0187 - accuracy: 0.1657 - val_loss: 6.0452 - val_accuracy: 0.1553
Epoch 5/100
1597/1597 [==============================] - 2s 1ms/step - loss: 5.9888 - accuracy: 0.1652 - val_loss: 6.0235 - val_accuracy: 0.1798
Epoch 6/100
1597/1597 [==============================] - 2s 1ms/step - loss: 5.9720 - accuracy: 0.1803 - val_loss: 6.0057 - val_accuracy: 0.1902
Epoch 7/100
1597/1597 [==============================] - 2s 1ms/step - loss: 5.9796 - accuracy: 0.1767 - val_loss: 6.0369 - va

1597/1597 [==============================] - 3s 2ms/step - loss: 5.9252 - accuracy: 0.1959 - val_loss: 6.0082 - val_accuracy: 0.1849
Epoch 58/100
1597/1597 [==============================] - 6s 4ms/step - loss: 5.9242 - accuracy: 0.1794 - val_loss: 6.0034 - val_accuracy: 0.1824
Epoch 59/100
1597/1597 [==============================] - 3s 2ms/step - loss: 5.9223 - accuracy: 0.1819 - val_loss: 6.0220 - val_accuracy: 0.1749
Epoch 60/100
1597/1597 [==============================] - 4s 2ms/step - loss: 5.9284 - accuracy: 0.1797 - val_loss: 6.0060 - val_accuracy: 0.1984
Epoch 61/100
1597/1597 [==============================] - 3s 2ms/step - loss: 5.9228 - accuracy: 0.1963 - val_loss: 6.0169 - val_accuracy: 0.2127
Epoch 62/100
1597/1597 [==============================] - 6s 4ms/step - loss: 5.9255 - accuracy: 0.1959 - val_loss: 6.0228 - val_accuracy: 0.2084
Epoch 63/100
1597/1597 [==============================] - 4s 3ms/step - loss: 5.9262 - accuracy: 0.2023 - val_loss: 5.9991 - val_accuracy

In [62]:
score = model.evaluate(X, yy, verbose=0)
print("Training Accuracy: {0:.2%}".format(score[1]))
score = model.evaluate(x_val, y_val, verbose=0)
print("Validation Accuracy: {0:.2%}".format(score[1]))
score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: {0:.2%}".format(score[1]))

Training Accuracy: 19.21%
Validation Accuracy: 19.96%
Testing Accuracy: 18.65%
